<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/computer%20vision%20algorithms%20part%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Edge detection algorithms

In [1]:
pip install opencv-python


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image_path = 'path_to_your_image.jpg'  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Unable to load image at {image_path}")
    exit()

# Sobel Edge Detection
def sobel_edge_detection(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    sobel_combined = cv2.magnitude(sobel_x, sobel_y)
    return sobel_x, sobel_y, sobel_combined

# Laplacian Edge Detection
def laplacian_edge_detection(image):
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    return laplacian

# Canny Edge Detection
def canny_edge_detection(image, low_threshold, high_threshold):
    canny = cv2.Canny(image, low_threshold, high_threshold)
    return canny

# Apply edge detection algorithms
sobel_x, sobel_y, sobel_combined = sobel_edge_detection(image)
laplacian = laplacian_edge_detection(image)
canny = canny_edge_detection(image, 100, 200)

# Plot the results
plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)
plt.title('Original Image')
plt.imshow(image, cmap='gray')

plt.subplot(2, 3, 2)
plt.title('Sobel X')
plt.imshow(sobel_x, cmap='gray')

plt.subplot(2, 3, 3)
plt.title('Sobel Y')
plt.imshow(sobel_y, cmap='gray')

plt.subplot(2, 3, 4)
plt.title('Sobel Combined')
plt.imshow(sobel_combined, cmap='gray')

plt.subplot(2, 3, 5)
plt.title('Laplacian')
plt.imshow(laplacian, cmap='gray')

plt.subplot(2, 3, 6)
plt.title('Canny')
plt.imshow(canny, cmap='gray')

plt.tight_layout()
plt.show()


HoughTransform

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError(f"Image not found at the path: {image_path}")
    return image

def detect_lines(image, canny_threshold1=50, canny_threshold2=150, hough_threshold=100):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, canny_threshold1, canny_threshold2)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, hough_threshold)
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

def detect_circles(image, dp=1.2, min_dist=100, param1=50, param2=30, min_radius=0, max_radius=0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp, min_dist,
                               param1=param1, param2=param2,
                               minRadius=min_radius, maxRadius=max_radius)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(image, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(image, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    return image

def display_image(image, title='Image'):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

def main():
    image_path = 'path_to_your_image.jpg'  # Replace with your image path
    image = load_image(image_path)

    line_detected_image = detect_lines(image.copy())
    display_image(line_detected_image, 'Hough Line Transform')

    circle_detected_image = detect_circles(image.copy())
    display_image(circle_detected_image, 'Hough Circle Transform')

if __name__ == "__main__":
    main()


OPtical flow

In [ ]:
import cv2
import numpy as np

def lucas_kanade_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7))

    mask = np.zeros_like(old_frame)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        good_new = p1[st == 1]
        good_old = p0[st == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
            frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

        img = cv2.add(frame, mask)

        cv2.imshow('Lucas-Kanade Optical Flow', img)
        if cv2.waitKey(30) & 0xFF == 27:
            break

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    cv2.destroyAllWindows()

def farneback_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    hsv_mask = np.zeros_like(frame1)
    hsv_mask[..., 1] = 255

    while True:
        ret, frame2 = cap.read()
        if not ret:
            break
        next_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prvs, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv_mask[..., 0] = angle * 180 / np.pi / 2
        hsv_mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

        bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

        cv2.imshow('Farneback Optical Flow', bgr)
        if cv2.waitKey(30) & 0xFF == 27:
            break

        prvs = next_frame

    cap.release()
    cv2.destroyAllWindows()

def main():
    video_path = 'path_to_your_video.mp4'  # Replace with your video path

    print("Running Lucas-Kanade Optical Flow")
    lucas_kanade_optical_flow(video_path)

    print("Running Farneback Optical Flow")
    farneback_optical_flow(video_path)

if __name__ == "__main__":
    main()


scale invariant feature

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image not found at the path: {image_path}")
    return image

def sift_feature_detection(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def draw_keypoints(image, keypoints):
    image_with_keypoints = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return image_with_keypoints

def match_features(descriptors1, descriptors2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def draw_matches(image1, keypoints1, image2, keypoints2, matches):
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return matched_image

def display_image(image, title='Image'):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def main():
    image_path1 = 'path_to_your_image1.jpg'  # Replace with your first image path
    image_path2 = 'path_to_your_image2.jpg'  # Replace with your second image path

    image1 = load_image(image_path1)
    image2 = load_image(image_path2)

    keypoints1, descriptors1 = sift_feature_detection(image1)
    keypoints2, descriptors2 = sift_feature_detection(image2)

    image1_with_keypoints = draw_keypoints(image1, keypoints1)
    image2_with_keypoints = draw_keypoints(image2, keypoints2)

    display_image(image1_with_keypoints, 'Image 1 with SIFT Keypoints')
    display_image(image2_with_keypoints, 'Image 2 with SIFT Keypoints')

    matches = match_features(descriptors1, descriptors2)

    matched_image = draw_matches(image1, keypoints1, image2, keypoints2, matches)

    display_image(matched_image, 'Matched Features between Images')

if __name__ == "__main__":
    main()
